In [44]:
#All labraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
import json
import re
import shutil

### part1

In [45]:
mkdir good_data_pred

In [46]:
mkdir bad_data_pred

In [47]:
schema_path='C://Users//hp//7.Muffassil_Project//3_Wafer_fault_detection//3_Muffassil_code/schema_prediction.json'
training_folder_path='C:\\Users\\hp\\7.Muffassil_Project\\3_Wafer_fault_detection\\3_Muffassil_code\\Prediction_Batch_files'

In [48]:
def valuesfromscheme():
    with open (schema_path, 'r') as f:
        data=json.load(f)   #conveting str to dict format ie we use json
#     print(type(data))
#     print(data)
    pattern=data['SampleFileName']
    LengthOfDateStampInFile=data['LengthOfDateStampInFile']
    LengthOfTimeStampInFile=data['LengthOfTimeStampInFile']
    NumberofColumns=data['NumberofColumns']
    
    return pattern,LengthOfDateStampInFile,LengthOfTimeStampInFile,NumberofColumns
    

In [49]:
pattern,LengthOfDateStampInFile,LengthOfTimeStampInFile,NumberofColumns=valuesfromscheme()

In [50]:
def regex():
    regex="['wafer_']+[\d_]+[\d]+\.csv"
    return regex

In [51]:
regex=regex()

In [52]:
def validationfilename (regex,LengthOfDateStampInFile,LengthOfTimeStampInFile):
    files=[i for i in os.listdir(training_folder_path)]
    for filename in files:
        if re.match(regex,filename):
            split_at_dot=re.split('.csv',filename)
            split_at_underline=re.split('_',split_at_dot[0])
            if len(split_at_underline[1])==LengthOfDateStampInFile:
                if len(split_at_underline[2])==LengthOfTimeStampInFile:
                    shutil.copy(f'{training_folder_path}\\{filename}','good_data_pred')
                else:
                    shutil.copy(f'{training_folder_path}\\{filename}','bad_data_pred')
                    
            else:
                    shutil.copy(f'{training_folder_path}\\{filename}','bad_data_pred')
                    
        else:
                    shutil.copy(f'{training_folder_path}\\{filename}','good_data_pred')

In [53]:
validationfilename (regex,LengthOfDateStampInFile,LengthOfTimeStampInFile)

In [54]:
# validating column length in the files

def validate_column (NumberofColumns):
    files=[i for i in os.listdir('good_data_pred') ]
    for filename in files:
        csv=pd.read_csv(f'good_data_pred/{filename}')
        if csv.shape[1]==NumberofColumns:
            pass
        else: 
            shutil.move(f'good_data_pred/{filename}','good_data_pred')

In [55]:
validate_column (NumberofColumns)

In [56]:
# Validate Missing Value in Whole Column

def Validate_missing_value_in_column():
    for file in os.listdir('good_data_pred'):
        csv=pd.read_csv(f'good_data_pred/{file}')
        count=0
        for column in csv:
            if csv[column].isnull().sum()==len(csv[column]):
#           if (len(csv[column])-csv[column].count())== len(csv[column]):
                count+=1
                shutil.move(f'good_data_pred/{file}','bad_data_pred')
                break
        if count==0:
            csv.rename(columns={'Unnamed: 0':'wafer'}, inplace=True)
            csv.to_csv(f'good_data_pred/{file}', index=None, header=True)
    
    print('[info] validation of column done')

In [57]:
Validate_missing_value_in_column()

[info] validation of column done


In [58]:
# combine all the file into 1 file
list=[]
for file in os.listdir('good_data_pred'):
    csv=pd.read_csv(f'good_data_pred/{file}')
    list.append(csv)
    
combined=pd.concat(list,axis=0, ignore_index=True)
combined.to_csv('pred_main_data.csv')



### part2

In [69]:
df=pd.read_csv('pred_main_data.csv')
df.head()

# remove unnamed column
df.drop(columns=['Unnamed: 0','wafer'], inplace=True)

from sklearn.impute import KNNImputer
imputer=KNNImputer(n_neighbors=3,missing_values=np.nan,)
new_df=imputer.fit_transform(df)

new_df=pd.DataFrame(new_df,columns=df.columns)


# checking the column which have a zero standard deviation
drop_col=[]
describe=new_df.describe()
for i in new_df.columns:
    if describe[i]['std']==0:
        drop_col.append(i)
print(drop_col)   

new_df.drop(columns=drop_col,inplace=True, axis=1)

new_df.to_csv('pred_clean_data.csv', index=False)

['Sensor-6', 'Sensor-14', 'Sensor-43', 'Sensor-50', 'Sensor-53', 'Sensor-70', 'Sensor-98', 'Sensor-142', 'Sensor-150', 'Sensor-179', 'Sensor-180', 'Sensor-187', 'Sensor-190', 'Sensor-191', 'Sensor-192', 'Sensor-193', 'Sensor-194', 'Sensor-195', 'Sensor-227', 'Sensor-230', 'Sensor-231', 'Sensor-232', 'Sensor-233', 'Sensor-234', 'Sensor-235', 'Sensor-236', 'Sensor-237', 'Sensor-238', 'Sensor-241', 'Sensor-242', 'Sensor-243', 'Sensor-244', 'Sensor-257', 'Sensor-258', 'Sensor-259', 'Sensor-260', 'Sensor-261', 'Sensor-262', 'Sensor-263', 'Sensor-264', 'Sensor-265', 'Sensor-266', 'Sensor-267', 'Sensor-277', 'Sensor-285', 'Sensor-314', 'Sensor-315', 'Sensor-316', 'Sensor-323', 'Sensor-326', 'Sensor-327', 'Sensor-328', 'Sensor-329', 'Sensor-330', 'Sensor-331', 'Sensor-365', 'Sensor-370', 'Sensor-371', 'Sensor-372', 'Sensor-373', 'Sensor-374', 'Sensor-375', 'Sensor-376', 'Sensor-379', 'Sensor-380', 'Sensor-381', 'Sensor-382', 'Sensor-395', 'Sensor-396', 'Sensor-397', 'Sensor-398', 'Sensor-399',

### part3

In [72]:
"""
putting prediction points into respective cluster based on the kmeans clustering model taht we used

"""


import joblib
data=pd.read_csv('pred_clean_data.csv')
model=joblib.load('models/kmeans.pkl')
y_pred=model.predict(data)
data['cluster']=y_pred

In [76]:
data.to_csv('pred_clusters.csv',index=False)

In [86]:
list_of_cluster=data['cluster'].unique()

#load all models

model0=joblib.load('models/Random_forest_0.pkl')
model1=joblib.load('models/xgboost_1.pkl')
model2=joblib.load('models/Random_forest_2.pkl')


for i in list_of_cluster:
    cluster_data=data[data['cluster']==i]
    
    cluster_data=cluster_data.iloc[:,:-1]
    
    if i==0:
        predict_0=model0.predict(cluster_data)
        
    elif i==1:
        predict_1=model1.predict(cluster_data)
    else:
        predict_2=model2.predict(cluster_data)
    

In [90]:
predict_0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [92]:
predict_2

array([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])